# MeanResponseTransformer
This notebook shows the functionality in the MeanResponseTransformer class. This transformer applies mean response encoding such that categorical levels are mapped to the average value of the response (target) for a particular problem.

In [30]:
import pandas as pd
import numpy as np
from pprint import pprint

In [31]:
import tubular
from tubular.nominal import MeanResponseTransformer

In [32]:
tubular.__version__

'0.2.11'

## Load Boston house price dataset from sklearn
Note, the load_boston script modifies the original Boston dataset to include nulls values and pandas categorical dtypes.

In [33]:
boston_df = tubular.testing.test_data.prepare_boston_df()
boston_df.shape

(506, 17)

In [34]:
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target,ZN_cat,CHAS_cat,RAD_cat
0,0.00632,18.0,2.31,0.0,0.538,6.575,NaN,4.0900,NaN,296.0,15.3,396.90,4.98,24.0,18.0,0.0,NaN
1,0.02731,NaN,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,NaN,0.0,2.0
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,NaN,17.8,392.83,4.03,34.7,0.0,0.0,2.0
3,NaN,NaN,2.18,0.0,0.458,NaN,45.8,6.0622,3.0,222.0,18.7,NaN,NaN,33.4,NaN,0.0,3.0
4,0.06905,0.0,2.18,0.0,0.458,NaN,NaN,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,0.0,0.0,3.0


In [35]:
boston_df.dtypes

CRIM         float64
ZN            object
INDUS        float64
CHAS          object
NOX          float64
RM           float64
AGE          float64
DIS          float64
RAD           object
TAX          float64
PTRATIO      float64
B            float64
LSTAT        float64
target       float64
ZN_cat      category
CHAS_cat    category
RAD_cat     category
dtype: object

## Simple usage

### Initialising MeanResponseTransformer
The `response_column` argument must be specified to set the response column that the fit method will use. <br>
There can be no nulls in the response column otherwise an exception will be raised.

In [36]:
mre_1 = MeanResponseTransformer(
    columns = 'CHAS', 
    response_column = 'target',
    copy = True, 
    verbose = True
)

BaseTransformer.__init__() called


### MeanResponseTransformer fit
The fit method must be run before the transform method. It calculates the average response column value for each level. The mappings are stored in an attribute called `mappings`.

In [37]:
mre_1.fit(boston_df)

BaseTransformer.fit() called


MeanResponseTransformer(columns=['CHAS'], response_column='target')

In [38]:
pprint(mre_1.mappings)

{'CHAS': {'0.0': 22.093842887473482, '1.0': 28.44}}


### MeanResponseTransformer transform

In [39]:
boston_df_2 = mre_1.transform(boston_df)

BaseTransformer.transform() called


In [40]:
boston_df_2['CHAS'].value_counts(dropna = False)

22.093843    471
28.440000     35
Name: CHAS, dtype: int64

## Transform with nulls
Null values are not converted in the MeanResponseTransformer. There are other transforrmers in the package which deal with imputation.

In [41]:
mre_2 = MeanResponseTransformer(
    columns = ['RAD', 'ZN_cat'], 
    response_column = 'target',
    copy = True, 
    verbose = True
)

BaseTransformer.__init__() called


In [42]:
mre_2.fit(boston_df)

BaseTransformer.fit() called


MeanResponseTransformer(columns=['RAD', 'ZN_cat'], response_column='target')

In [43]:
boston_df[['RAD', 'ZN_cat']].isnull().sum()

RAD       62
ZN_cat    62
dtype: int64

In [44]:
try:
    boston_df_3 = mre_2.transform(boston_df)
except Exception as e:
    print(e)

nulls would be introduced into column RAD from levels not present in mapping


## Weights column
It is possible to specify a weights column using the `weights_column` argument when initialising the transformer. <br>
If this is the case then a weighted mean, using this column, will be calculated in `fit`.

In [45]:
mre_3 = MeanResponseTransformer(
    columns = 'CHAS', 
    response_column = 'target',
    weights_column = 'CRIM'
)

In [46]:
mre_3.fit(boston_df)

MeanResponseTransformer(columns=['CHAS'], response_column='target',
                        weights_column='CRIM')

In [47]:
boston_df_4 = mre_3.transform(boston_df)

In [48]:
boston_df_4['CHAS'].value_counts(dropna = False)

6.371817     471
17.690223     35
Name: CHAS, dtype: int64